In [145]:
from pynq import Overlay
ol = Overlay("/home/xilinx/jupyter_notebooks/PYNQ_Dev/VinDevData/HAPI/hapi-des/des_hapi.bit")

ol.download()
ol.bitstream.timestamp

from pynq import PL
PL.bitfile_name

ol.is_loaded()

True

In [146]:
from pynq import MMIO
  
class hapiDES:
    """Brief description of Module goes here.
    
    Attributes
    ----------
    data : List of size2 (int or hex values) 
       Input Data for Encryption or Decryption: data[0] is MSB 32 bits and data [1] is LSB 32 bits
    result_ENC_DEC : List of size2 (hex value) 
       Return results of Encryption or Decryption: result_ENC_DEC[0] is MSB 32 bits and result_ENC_DEC[1] is LSB 32 bits
       
   """
    mmio = MMIO(0x43C40000,0x00010000)
    global result_ENC_DEC
    result_ENC_DEC=[0,0]

    def __init__(self):
        self.mmio.write(0, 0)
     
    def reset_des_accel(self):
        config_reg=0x80000001
        mmio.write(0,config_reg)
        print("DES ACCELERATOR RESET")
        config_reg=0x80000001
        mmio.write(0,config_reg)

    def des_status(self):
        config_reg=mmio.read(0)
        print("DES ACCEL STATUS")
        if config_reg==0x80000001:
            print("RESET")
            print("MODE:ENCRYPT")
        elif config_reg==0x80000000:
            print("RESET")
            print("MODE:DECRYPT")
        elif config_reg==0x00000000:
            print("ACTIVE (NOT IN RESET)")
            print("MODE:DECRYPT")
        elif config_reg==0x00000001:
            print("ACTIVE (NOT IN RESET)")
            print("MODE:ENCRYPT")
        else:
            print("ERROR: NOT CONFIGURED PROPERLY --> SOMETHING IS WRONG")
    
    def set_oper_encrypt(self):
        config_reg=0x00000001
        mmio.write(0,config_reg)
        print("DES ACCELERATOR IS SET TO ENCRYPT")
        print("-->SET A NEW KEY OR WILL USE THE PREVIOUS SET KEY")
        
    def set_oper_decrypt(self):
        config_reg=0x00000000
        mmio.write(0,config_reg)
        print("DES ACCELERATOR IS SET TO DECRYPT")
        print("-->SET A NEW KEY OR WILL USE THE PREVIOUS SET KEY")

    def result(self):     
        result_ENC_DEC[0]= hex(mmio.read(20))
        result_ENC_DEC[1]= hex(mmio.read(24))
        return result_ENC_DEC
    
    def set_key(self,data):
        mmio.write(12,data[0])
        mmio.write(16,data[1])
        print("CRYPTO KEY UPDATED")
 
    def encrypt(self,data):    
        mmio.write(4,data[0])
        mmio.write(8,data[1])
        self.result()
        return result_ENC_DEC
    
    def decrypt(self,data):
        mmio.write(4,data[0])
        mmio.write(8,data[1])
        self.result()
        return result_ENC_DEC
 

In [147]:
import time
import math
from pynq import MMIO
mmio = MMIO(0x43C40000,0x00010000)
des=hapiDES()
des.reset_des_accel()
des.set_oper_encrypt()
MyKey=[0xfdcb0,0x167cde]
des.set_key(MyKey)
input_data=[0xFE123410,0x12354567]
des.encrypt(input_data)

DES ACCELERATOR RESET
DES ACCELERATOR IS SET TO ENCRYPT
-->SET A NEW KEY OR WILL USE THE PREVIOUS SET KEY
CRYPTO KEY UPDATED


['0xdbd44175', '0x8cd6a970']

In [148]:
import re
import unicodedata
def data_formatter(txt):
    dataIn=''.join(hex(ord(x))[2:] for x in txt)
    #print(dataIn)
    print(len(dataIn))
    vin=[]
    for i in range(0,len(dataIn),8):
        vin.append(''.join(map(str,dataIn[i:i+8])))
    vin=['0x' + s for s in vin]
    print("INPUT")
    print(vin)
    vin_bc=vin
    vin=[int(x,16) for x in vin]
    if len(vin)%2!=0:
        vin.append(0)
    return vin,vin_bc

#txt="""I don't know why I do it...
#I've nothing profound to say!
#But late in the Night ..
#I sit in the light..
#Or Rather should I say
#Watching rainbows around me, I scribble and scribble away..."""

txt ="Let us try to encrypt this"
data_2send,enc_in_hex=data_formatter(txt)
print(enc_in_hex)
cryptRes=[]
j=0
TOTAL_BLOCKS=len(data_2send)
start_time = time.time()
for i in range(0,len(data_2send),2):
    cryptRes[j:j+1]=des.encrypt(data_2send[i:i+2])
    j=j+2
exec_time=time.time() - start_time
print("--- %s seconds ---" % (exec_time))
print("--- %s Blocks Per Second---" % ((math.pow(1, 1))*(TOTAL_BLOCKS/2)/exec_time))
print(cryptRes)

52
INPUT
['0x4c657420', '0x75732074', '0x72792074', '0x6f20656e', '0x63727970', '0x74207468', '0x6973']
['0x4c657420', '0x75732074', '0x72792074', '0x6f20656e', '0x63727970', '0x74207468', '0x6973']
--- 0.002146005630493164 seconds ---
--- 1863.9280079991113 Blocks Per Second---
['0x176c295', '0xd1470f4a', '0x6e8c69a9', '0xc3023084', '0x845b63ce', '0x9ec95823', '0xdedbc128', '0xb0042c0b']


In [149]:
des.set_oper_decrypt()
decipher=[]
j=0
TOTAL_BLOCKS=len(cryptRes)
cryptRes=[int(x,16) for x in cryptRes]
#print(cryptRes)
start_time = time.time()
for i in range(0,len(cryptRes),2):
    decipher[j:j+1]=des.decrypt(cryptRes[i:i+2])
    j=j+2
exec_time=time.time() - start_time
print("--- %s seconds ---" % (exec_time))
print("--- %s Blocks Per Second---" % ((math.pow(1, 1))*(TOTAL_BLOCKS/2)/exec_time))
print(decipher)

DES ACCELERATOR IS SET TO DECRYPT
-->SET A NEW KEY OR WILL USE THE PREVIOUS SET KEY
--- 0.002225637435913086 seconds ---
--- 1797.2379217996786 Blocks Per Second---
['0x4c657420', '0x75732074', '0x72792074', '0x6f20656e', '0x63727970', '0x74207468', '0x6973', '0x0']


In [150]:
#bool(set(decipher) & set(enc_in_hex))
print(set(decipher)^set(enc_in_hex))
#print(x) for x,y in (decipher,enc_in_hex) if x!=y

{'0x0'}


In [151]:
dec=(''.join(map(str,decipher)))
dec=dec.replace("0x","")
print(dec)
ascii_string =''
x = 0
y = 2
l = len(dec)
while y <= l:
    ascii_string += chr(int(dec[x:y], 16))
    x += 2
    y += 2
print (ascii_string)

4c65742075732074727920746f20656e637279707420746869730
Let us try to encrypt this
